In [1]:
from IPython.display import HTML, display

display(HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
toggle the code of this notebook on/off &nbsp;<a href="javascript:code_toggle()">^</a>'''))

### PANORAMAS OF CINEMA online

# // IMAGE SEARCH
v0.02

In [2]:
# REQUEST INDEXES
import requests
r = requests.get("https://panoramas-of-cinema.s3.eu-central-1.amazonaws.com/indexes/videotheques_inv.json")
query_response = r.json()

button_color = 'black' # #7d1b27 darkRed

## SEARCH THE VIDEOTHEQUE

### places index

In [3]:
# SHOW INDEXES 
for e in sorted(query_response['places']):
    print(e, end='  .  ')

airfield  .  airplane_cabin  .  airport_terminal  .  alcove  .  alley  .  amphitheater  .  amusement_arcade  .  amusement_park  .  apartment_building_outdoor  .  aquarium  .  aqueduct  .  arcade  .  arch  .  archaelogical_excavation  .  archive  .  arena_hockey  .  arena_performance  .  arena_rodeo  .  army_base  .  art_gallery  .  art_school  .  art_studio  .  artists_loft  .  assembly_line  .  athletic_field_outdoor  .  atrium_public  .  attic  .  auditorium  .  auto_factory  .  auto_showroom  .  badlands  .  bakery_shop  .  balcony_exterior  .  balcony_interior  .  ball_pit  .  ballroom  .  bamboo_forest  .  bank_vault  .  banquet_hall  .  bar  .  barn  .  barndoor  .  baseball_field  .  basement  .  basketball_court_indoor  .  bathroom  .  bazaar_indoor  .  bazaar_outdoor  .  beach  .  beach_house  .  beauty_salon  .  bedchamber  .  bedroom  .  beer_garden  .  beer_hall  .  berth  .  biology_laboratory  .  boardwalk  .  boat_deck  .  boathouse  .  bookstore  .  booth_indoor  .  bot

### objects index

In [4]:
# SHOW INDEXES 
for e in sorted(query_response['objects']):
    print(e, end='  .  ')

aeroplane  .  apple  .  backpack  .  banana  .  baseball_bat  .  baseball_glove  .  bear  .  bed  .  bench  .  bicycle  .  bird  .  boat  .  book  .  bottle  .  bowl  .  broccoli  .  bus  .  cake  .  car  .  carrot  .  cat  .  cell_phone  .  chair  .  clock  .  cow  .  cup  .  diningtable  .  dog  .  donut  .  elephant  .  fire_hydrant  .  fork  .  frisbee  .  giraffe  .  hair_drier  .  handbag  .  horse  .  hot_dog  .  keyboard  .  kite  .  knife  .  laptop  .  microwave  .  motorbike  .  mouse  .  orange  .  oven  .  parking_meter  .  person  .  pizza  .  pottedplant  .  refrigerator  .  remote  .  sandwich  .  scissors  .  sheep  .  sink  .  skateboard  .  skis  .  snowboard  .  sofa  .  spoon  .  sports_ball  .  stop_sign  .  suitcase  .  surfboard  .  teddy_bear  .  tennis_racket  .  tie  .  toaster  .  toilet  .  toothbrush  .  traffic_light  .  train  .  truck  .  tvmonitor  .  umbrella  .  vase  .  wine_glass  .  zebra  .  

### > type the indexes and tune the confidences

In [5]:
# DISPLAY WIDGETS 
from IPython.display import display, clear_output
from ipywidgets import interactive, interact
import ipywidgets as widgets

options = []
s=0.01
for i in range(100):
    options.append("%.2f"%s)
    s+=.01

def p(place, confidence):
    return

def o(obj, confidence):
    return

w_places = interactive(p, place='',
                       confidence = widgets.SelectionRangeSlider(options=options, index=(0,99), description='confidence') )

w_objects = interactive(o, obj='',
                        confidence = widgets.SelectionRangeSlider(options=options, index=(0,99), description='confidence') )


display(w_places)
display(w_objects)

interactive(children=(Text(value='', description='place'), SelectionRangeSlider(description='confidence', inde…

interactive(children=(Text(value='', description='obj'), SelectionRangeSlider(description='confidence', index=…

### > select the faces

In [6]:
def f(no_faces, yes_faces):
    return
w_faces = interactive(f, no_faces=True, yes_faces=True)
display(w_faces)

interactive(children=(Checkbox(value=True, description='no_faces'), Checkbox(value=True, description='yes_face…

In [7]:
# REFRESH QUERY
def refresh_query(event):
    clear_output()
    display(refresh_button)
    
    place = w_places.kwargs['place']
    #place_conf = "%s,%s"%(w_places.kwargs['min_conf'],w_places.kwargs['max_conf'])
    place_conf = "%s,%s"%(w_places.kwargs['confidence'])

    obj = w_objects.kwargs['obj']
    #obj_conf = "%s,%s"%(w_objects.kwargs['min_conf'],w_objects.kwargs['max_conf'])
    obj_conf = "%s,%s"%(w_objects.kwargs['confidence'])

    faces = str()
    if w_faces.kwargs['no_faces']:
        faces = faces+'no_faces'
    if w_faces.kwargs['yes_faces']:
        if len(faces) > 0:
            faces = faces+',small_faces'
        else:
            faces = faces+'small_faces'

    # 
    this_query = 'http://pocquery-env.eba-ap3n3cr3.eu-central-1.elasticbeanstalk.com/query?'
    if place: this_query = this_query+'places=%s,%s'%(place,place_conf)
    if faces: this_query = this_query+'&faces=%s'%faces
    if obj: this_query = this_query+'&objects=%s,%s'%(obj,obj_conf)
    
    global my_frames
    my_frames = requests.get(this_query).json()['response']
    print (len(my_frames), 'images with this search') # this_query[73:]
    
    global my_globals
    my_globals = {'plotted':[], 'w_buttons':[]}

                    
refresh_button = widgets.Button(description="search", button_style='info')
refresh_button.style.button_color = button_color
display(refresh_button)
refresh_button.on_click(refresh_query)

Button(button_style='info', description='search', style=ButtonStyle(button_color='black'))

In [8]:
# PLOT RANDOM FRAMES
def plot_frames(event):
    clear_output()
    display(plot_button)
    
    how_many = 16
    plot_x, plot_y = 4,4

    if len(my_frames) < how_many:
        how_many = len(my_frames)

    all_i = [ i for i in range(len(my_frames)) ]

    import random
    on_off = True
    this_sample = []
    while on_off:
        if len(my_globals['plotted']) == len(my_frames):
            my_globals['plotted'] = []
            print("...you've seen it all")
        r = random.choice(all_i)
        if r in my_globals['plotted']:
            pass
        else:
            this_sample.append(r)
            my_globals['plotted'].append(r)
            if len(my_globals['plotted']) == len(my_frames):
                print('...last batch')
                break
        if len(this_sample) == how_many:
            break

    # OPEN IMAGES ------------------------------
    import requests
    s3BucketName = "panoramas-of-cinema"
    region_name = 'eu-central-1'

    i=0
    frames_plot = []
    for e in my_frames:
        film = e['movieId']
        frame = e['frameId']
        if i in this_sample:
            try:
                objectKey = "frames_db%s/by_scene_thumbnail%s" % (film, frame)
                this_frame = "https://%s.s3.%s.amazonaws.com/%s"%(s3BucketName,region_name,objectKey)
                img = requests.get(this_frame).content          
                frames_plot.append( (film, frame, img) )
            except:
                raise 
        i+=1

    # PLOT ------------------------------
    def on_button_clicked(event):
        global movie_2p
        global frame_2p
        this_button = id(event)
        G = globals()
        all_buttons = G['my_globals']['w_buttons']
        for i, b in enumerate(all_buttons):
            this_id = id(b)
            if this_button == this_id:
                movie_2p = my_globals['w_buttons'][i].description
                frame_2p = my_globals['w_buttons'][i].tooltip

    print('...plotting %s images from %s'%(how_many, len(my_frames)))
    from ipywidgets import Button, HBox, VBox
    import ipywidgets as widgets

    k = 0
    movie_index, movie_names = 0, []
    vboxes = []
    for i in range(plot_x):
        this_col = []
        for j in range(plot_y):
            if k < len(frames_plot):
                try:
                    image_w = widgets.Image(value=frames_plot[k][2], format='png', width=250) #height=400
                    button = widgets.Button(description=frames_plot[k][0], tooltip=frames_plot[k][1])
                    my_globals['w_buttons'].append(button)

                    button.on_click( on_button_clicked )

                    this_box = [image_w, button]
                    this_col = this_col+this_box

                except:
                    raise
            k+=1

        box = VBox( this_col )
        vboxes.append(box)

    display(HBox( vboxes ))
    
plot_button = widgets.Button(description="show", button_style='info')
plot_button.style.button_color = button_color
display(plot_button)
plot_button.on_click(plot_frames)

Button(button_style='info', description='show', style=ButtonStyle(button_color='black'))

In [9]:
print('\n', '\n', '\n')

## HD IMAGE

### > click on an image

In [10]:
from IPython.display import Image, display

def get_hq_image(event):
    clear_output()
    display(hq_image_button)
    
    objectKey = "frames_db%s/by_scene%s" % (movie_2p, frame_2p)
    display(Image(url='https://panoramas-of-cinema.s3.eu-central-1.amazonaws.com/%s'%objectKey, width=450))
    print('%s%s'%(movie_2p[1:],frame_2p[6:-4]))
    
hq_image_button = widgets.Button(description="hq image", button_style='info')
hq_image_button.style.button_color = button_color
display(hq_image_button)
hq_image_button.on_click(get_hq_image)

Button(button_style='info', description='hq image', style=ButtonStyle(button_color='black'))

In [11]:
print('\n', '\n', '\n')

## PLAY A CLIP

### > click on an image

In [12]:
# PLAY THE CLIP 
def play_clip(event):
    clear_output()
    display(ui1)

    global r
    clip_query = 'http://pocgetclip-env.eba-4phq6b4m.eu-central-1.elasticbeanstalk.com/clip?movie=%s&frame=%s'%(movie_2p[1:],frame_2p[7:-4])
    r = requests.get(clip_query).json()

    from IPython.display import Video
    display(Video(r['play'], height=300, html_attributes="controls autoplay"))
    print('%s%s'%(movie_2p[1:],frame_2p[6:-4]))
    
def play_before(event):
    clear_output()
    display(ui1)
    
    from IPython.display import Video
    display(Video(r['before'], height=300, html_attributes="controls autoplay"))
    this_f = r['before'].find('frame-')
    print('%s%s'%(movie_2p[1:],r['before'][(this_f)+5:-4]))
    
def play_after(event):
    clear_output()
    display(ui1)
    
    from IPython.display import Video
    display(Video(r['after'], height=300, html_attributes="controls autoplay"))
    this_f = r['after'].find('frame-')
    print('%s%s'%(movie_2p[1:],r['after'][(this_f)+5:-4]))
    
# buttons box
play_button = widgets.Button(description="play", button_style='info')
play_button.style.button_color = button_color
playB_button = widgets.Button(description="<<<", button_style='info')
playB_button.style.button_color = button_color
playA_button = widgets.Button(description=">>>", button_style='info')
playA_button.style.button_color = button_color
ui1 = widgets.HBox([playB_button, play_button, playA_button])
display(ui1)


play_button.on_click(play_clip)
playB_button.on_click(play_before)
playA_button.on_click(play_after)

In [13]:
print('\n', '\n', '\n')

## YOUR TAGS

### ADD

### > click on an image and type its tag and your user name

In [14]:
tag_http = 'http://poctag-env.eba-vjpstfpz.eu-central-1.elasticbeanstalk.com/tags'

In [15]:
# DISPLAY WIDGETS 
def getUser(user):
    return

def getTag(tag):
    return

w_user = interactive(getUser, user='')
w_tag = interactive(getTag, tag='')

# REFRESH QUERY
def add_tag(event):
    clear_output()
    display(w_tag)
    display(w_user)
    display(add_tag_button)
    
    this_user = w_user.kwargs['user']
    this_tag = w_tag.kwargs['tag']
    this_frame = '%s,%s'%(movie_2p,frame_2p)
    this_request = tag_http+'?action=ADD&tag=%s&user=%s&frame=%s'%(this_tag,this_user,this_frame)
    r = requests.get(this_request).json()['statusCode']
    if r == 200: print('tag added to : %s %s'%(movie_2p, frame_2p))
    else: print ('error %s'%r)

add_tag_button = widgets.Button(description="add tag", button_style='info')
add_tag_button.style.button_color = button_color
add_tag_button.on_click(add_tag)

display(w_tag)
display(w_user)
display(add_tag_button)

interactive(children=(Text(value='', description='tag'), Output()), _dom_classes=('widget-interact',))

interactive(children=(Text(value='', description='user'), Output()), _dom_classes=('widget-interact',))

Button(button_style='info', description='add tag', style=ButtonStyle(button_color='black'))

In [16]:
print('\n', '\n', '\n')

### SEARCH

### tags index

In [17]:
this_request = tag_http+'?action=GET_ALL_TAGS'
r = requests.get(this_request).json()['body']
for e in sorted(r):
    print(e, end='  .  ')

def get_tags(event):
    clear_output()

    this_request = tag_http+'?action=GET_ALL_TAGS'
    r = requests.get(this_request).json()['body']
    for e in sorted(r):
        print(e, end='  .  ')
    
    display(refresh_tags_button)

        
refresh_tags_button = widgets.Button(description="refresh tags", button_style='info')
refresh_tags_button.style.button_color = button_color
display(refresh_tags_button)
refresh_tags_button.on_click(get_tags)

 mouth  .  Archive  .  Elizabeth  .  H2O  .  Marchant  .  Na  .  Sophists’ money  .  Spirit  .  agora  .  agora portal  .  agora theatre  .  aiakos  .  aisle  .  alien  .  all  .  amphitheatre  .  apollo  .  apollon  .  apotheosium  .  applause  .  arava_valley  .  arcade  .  architecturalcomponent  .  archive  .  area  .  area_lizzy  .  area_lizzy2  .  arena  .  ark  .  assembly  .  atmosphere  .  attack  .  audience  .  auditorium  .  balcony  .  balloon  .  ballroom  .  bank  .  banquet  .  banquet panorama  .  banquet2  .  banquetpanorama  .  bar  .  baroque  .  basileion  .  basileios  .  basilica  .  basilica_altar  .  basilica_atmosphere  .  basilica_ceiling  .  basilica_columns  .  basilica_floor  .  basin  .  bath  .  bath   .  bathrobe  .  bderoom  .  beach  .  bedroom  .  bedroom_atmosphere  .  bema  .  big_hall  .  billboard  .  biosphere  .  bird  .  bistrot_atmosphere  .  bistrot_ceiling  .  bistrot_columns  .  bistrot_floor  .  bistrot_table  .  bite  .  black  .  blacko

Button(button_style='info', description='refresh tags', style=ButtonStyle(button_color='black'))

### users index

In [18]:
this_request = tag_http+'?action=GET_ALL_USERS'
r = requests.get(this_request).json()['body']
for e in sorted(r):
    print(e, end='  .  ')

def get_users(event):
    clear_output()

    this_request = tag_http+'?action=GET_ALL_USERS'
    r = requests.get(this_request).json()['body']
    for e in sorted(r):
        print(e, end='  .  ')
    
    display(refresh_users_button)

        
refresh_users_button = widgets.Button(description="refresh users", button_style='info')
refresh_users_button.style.button_color = button_color
display(refresh_users_button)
refresh_users_button.on_click(get_users)

3-2  .  Helia  .  Julie  .  Rachelle  .  archive  .  birk  .  circe  .  cowboy  .  dominic  .  eva  .  flavio  .  gaad  .  gsm  .  h  .  heldc  .  helia  .  hero  .  jb  .  jno  .  jorge  .  julie  .  julie   .  k  .  lah  .  laura  .  liquid  .  oliver  .  page  .  poureli  .  queenlizzy  .  sebastian  .  secret  .  sigi  .  slash  .  sophie  .  stan  .  toti  .  vampire  .  wayward  .  

Button(button_style='info', description='refresh users', style=ButtonStyle(button_color='black'))

### > type a tag and/or user

In [19]:
# BOXES 
def getUserSearch(user_search):
    return

def getTagSearch(tag_search):
    return

w_user_search = interactive(getUserSearch, user_search='')
w_tag_search = interactive(getTagSearch, tag_search='')

# BUTTON
def search_tag(event):
    clear_output()
    display(w_tag_search)
    display(w_user_search)
    display(search_tag_button)
    
    this_user = w_user_search.kwargs['user_search']
    this_tag = w_tag_search.kwargs['tag_search']
    
    global my_frames_tag
    if len(this_user) > 0 and len(this_tag) > 0:
        this_request = tag_http+'?action=GET_FRAMES_USER_TAG&tag=%s&user=%s'%(this_tag,this_user)
        my_frames_tag = requests.get(this_request).json()
        this_length = len(my_frames_tag['body'])
        print (this_length, 'images with this search')
    elif len(this_user) > 0 and len(this_tag) == 0:
        this_request = tag_http+'?action=GET_FRAMES_USER&user=%s'%(this_user)
        my_frames_tag = requests.get(this_request).json()
        this_length = len(my_frames_tag['body'])
        print (this_length, 'images with this search')
        this_tags = set([ f['tag'] for f in my_frames_tag['body'] ])
        print (this_user, 'is talking about  .  ', end='')
        for e in sorted(this_tags): print(e, end='  .  ')
    elif len(this_user) == 0 and len(this_tag) > 0:
        this_request = tag_http+'?action=GET_FRAMES_TAG&tag=%s'%(this_tag)
        my_frames_tag = requests.get(this_request).json()
        this_length = len(my_frames_tag['body'])
        print (this_length, 'images with this search')
        this_users = set([ f['user'] for f in my_frames_tag['body'] ])
        print (this_tag, 'is talked about by  .  ', end='')
        for e in sorted(this_users): print(e, end='  .  ')
    else:
        print ('type a user name and/or a tag')
                
    global my_globals_tag
    my_globals_tag = {'plotted':[], 'w_buttons':[]}
        
search_tag_button = widgets.Button(description="search tag", button_style='info')
search_tag_button.style.button_color = button_color
search_tag_button.on_click(search_tag)

display(w_tag_search)
display(w_user_search)
display(search_tag_button)

interactive(children=(Text(value='', description='tag_search'), Output()), _dom_classes=('widget-interact',))

interactive(children=(Text(value='', description='user_search'), Output()), _dom_classes=('widget-interact',))

Button(button_style='info', description='search tag', style=ButtonStyle(button_color='black'))

In [20]:
# PLOT RANDOM FRAMES 
def plot_frames_tags(event):
    clear_output()
    display(plot_button_tags)
    
    my_frames = my_frames_tag['body']
    how_many = 16
    plot_x, plot_y = 4,4

    if len(my_frames) < how_many:
        how_many = len(my_frames)

    all_i = [ i for i in range(len(my_frames)) ]

    import random
    on_off = True
    this_sample = []
    while on_off:
        if len(my_globals_tag['plotted']) == len(my_frames):
            my_globals_tag['plotted'] = []
            print("...you've seen it all")
        r = random.choice(all_i)
        if r in my_globals_tag['plotted']:
            pass
        else:
            this_sample.append(r)
            my_globals_tag['plotted'].append(r)
            if len(my_globals_tag['plotted']) == len(my_frames):
                print('...last batch')
                break
        if len(this_sample) == how_many:
            break

    # OPEN IMAGES ------------------------------
    import requests

    s3BucketName = "panoramas-of-cinema"
    region_name = 'eu-central-1'

    i=0
    frames_plot = []
    for e in my_frames:
        film = e['movie']
        frame = e['frame']
        if i in this_sample:
            try:
                objectKey = "frames_db%s/by_scene_thumbnail%s" % (film, frame)
                this_frame = "https://%s.s3.%s.amazonaws.com/%s"%(s3BucketName,region_name,objectKey)
                img = requests.get(this_frame).content          
                frames_plot.append( (film, frame, img) )
            except:
                raise 
        i+=1

    # PLOT ------------------------------
    def on_button_clicked(event):
        global movie_2p_tag, frame_2p_tag
        global movie_2p, frame_2p
        
        this_button = id(event)
        G = globals()
        all_buttons = G['my_globals_tag']['w_buttons']
        for i, b in enumerate(all_buttons):
            this_id = id(b)
            if this_button == this_id:
                movie_2p_tag = my_globals_tag['w_buttons'][i].description
                frame_2p_tag = my_globals_tag['w_buttons'][i].tooltip
                movie_2p = my_globals_tag['w_buttons'][i].description
                frame_2p = my_globals_tag['w_buttons'][i].tooltip
                
    print('...plotting %s images from %s'%(how_many, len(my_frames)))
    from ipywidgets import Button, HBox, VBox
    import ipywidgets as widgets

    k = 0
    movie_index, movie_names = 0, []
    vboxes = []
    for i in range(plot_x):
        this_col = []
        for j in range(plot_y):
            if k < len(frames_plot):
                try:
                    image_w = widgets.Image(value=frames_plot[k][2], format='png', width=250) #height=400
                    button = widgets.Button(description=frames_plot[k][0], tooltip=frames_plot[k][1])
                    my_globals_tag['w_buttons'].append(button)

                    button.on_click( on_button_clicked )

                    this_box = [image_w, button]
                    this_col = this_col+this_box

                except:
                    raise
            k+=1

        box = VBox( this_col )
        vboxes.append(box)

    display(HBox( vboxes ))
    
plot_button_tags = widgets.Button(description="show", button_style='info')
plot_button_tags.style.button_color = button_color
display(plot_button_tags)
plot_button_tags.on_click(plot_frames_tags)

Button(button_style='info', description='show', style=ButtonStyle(button_color='black'))

In [21]:
print('\n', '\n', '\n')

### DELETE

### > click on an image

In [22]:
def delete_tag(event):
    clear_output()
    display(delete_tag_button)
        
    this_user = w_user_search.kwargs['user_search']
    this_tag = w_tag_search.kwargs['tag_search']
    if len(this_user) > 0 and len(this_tag) > 0:
        this_frame = '%s,%s'%(movie_2p_tag,frame_2p_tag)
        this_request = tag_http+'?action=DELETE&tag=%s&user=%s&frame=%s'%(this_tag,this_user,this_frame)
        r = requests.get(this_request).json()['statusCode']
        if r == 200: print('tag deleted : %s %s'%(movie_2p_tag, frame_2p_tag))
        else: print ('error %s'%r)
    else:
        print ('! user or tag is missing')

    
delete_tag_button = widgets.Button(description="delete", button_style='info')
delete_tag_button.style.button_color = button_color
display(delete_tag_button)
delete_tag_button.on_click(delete_tag)

Button(button_style='info', description='delete', style=ButtonStyle(button_color='black'))

In [23]:
print('\n', '\n', '\n', '\n', '\n')

ETH Zurich &nbsp;&nbsp;&nbsp;&nbsp; Department of Architecture &nbsp;&nbsp;&nbsp;&nbsp; Chair for Digital Architectonics &nbsp;&nbsp;&nbsp;&nbsp; 2021